### Scraping processes

In [11]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import os
import pandas as pd
from datetime import datetime

In [12]:
ads_type = 0
estate_type =0
region = 0

while True:
    ads_type = input("Zəhmət olmasa nümunə formasında elan növünü qeyd edin: 1-Alış/ 2-Kirayə")
    try:
        ads_type =int(ads_type)
        if ads_type in (1,2):
            break 
    except ValueError:
            print("Daxil edilən məlumatda uyğunsuzluq var. Yenidən daxil edin!")
            
while True:
    estate_type = input("Zəhmət olmasa nümunə formasında əmlak növünü qeyd edin: 1-Mənzil/2-Yeni tikili/3-Köhnə tikili/4-Ev-villa/5-Bağ")
    try:
        estate_type =int(estate_type)
        if estate_type in (1,2,3,4,5):
            break 
    except ValueError:
            print("Daxil edilən məlumatda uyğunsuzluq var. Yenidən daxil edin!")        
            
while True:
    region = input("Zəhmət olmasa nümunə formasında ərazini qeyd edin: 1-Nərimanov r/2-Nizami r/3-Yasamal r")
    try:
        region =int(region)
        if region in (1,2,3):
            break 
    except ValueError:
            print("Daxil edilən məlumatda uyğunsuzluq var. Yenidən daxil edin!")  
            
while True:
    elan_sayi = input("Zəhmət olmasa istədiyiniz minimum elan sayını ədədlə qeyd edin: ")
    try:
        elan_sayi =int(elan_sayi)
        break
    except ValueError:
            print("Daxil edilən sayda uyğunsuzluq var. Yenidən rəqəm formasında daxil edin!")        

Zəhmət olmasa nümunə formasında elan növünü qeyd edin: 1-Alış/ 2-Kirayə 1
Zəhmət olmasa nümunə formasında əmlak növünü qeyd edin: 1-Mənzil/2-Yeni tikili/3-Köhnə tikili/4-Ev-villa/5-Bağ 1
Zəhmət olmasa nümunə formasında ərazini qeyd edin: 1-Nərimanov r/2-Nizami r/3-Yasamal r 1
Zəhmət olmasa istədiyiniz minimum elan sayını ədədlə qeyd edin:  15


In [13]:
#Check ad type
if ads_type == 1:
    ads_type = "Alış"
if ads_type == 2:
    ads_type = "Kirayə"
    
#Check estate type  
if estate_type == 1:
    estate_type = "Mənzil"
if estate_type == 2:
    estate_type = "Yeni tikili"
if estate_type == 3:
    estate_type = "Köhnə tikili"
if estate_type == 4:
    estate_type = "Ev / villa"
if estate_type == 5:
    estate_type = "Bağ"
    
#Check region
if region == 1:
    region = "Nərimanov r."
if region == 2:
    region = "Nizami r."
if region == 3:
    region = "Yasamal r."
print(f"Elan növü: {ads_type}; Əmlak növü: {estate_type}; Region: {region}")

#Set ad count
ad_count = elan_sayi


Elan növü: Alış; Əmlak növü: Mənzil; Region: Nərimanov r.


In [14]:
class BinaAz:    
    def __init__(self,ads_type,estate_type,region,ad_count):
        self.service = Service(executable_path='chromedriver.exe')
        self.browserProfile = webdriver.ChromeOptions()
        self.browserProfile.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})
        self.browser = webdriver.Chrome(service=self.service, options=self.browserProfile)

        self.browser = webdriver.Chrome()
        self.ads_type=ads_type
        self.estate_type=estate_type
        self.region=region
        self.ad_count =ad_count
        self.id_set=set()
        self.ad_info_list =[]
        self.browser.get("https://bina.az/")
        time.sleep(0.5)
        self.browser.find_element("xpath","//*[@id='new_q']/div[1]/div[1]/div[1]/div[2]").click()
        self.browser.find_element("xpath",f"//span[.='{self.ads_type}']").click()
        time.sleep(0.5)
        self.browser.find_element("xpath","//*[@id='new_q']/div[1]/div[2]/div[1]/div[2]").click()
        time.sleep(0.5)
        self.browser.find_element("xpath",f"//span[.='{self.estate_type}']").click()
        time.sleep(1)
        self.browser.find_element("xpath"," //*[@id='js-search-row-filters-btn']").click()
        time.sleep(4)
        self.browser.find_element("xpath","//*[@id='js-search-filters-row-locations']/div[2]/a[1]").click()
        time.sleep(1)
        self.browser.find_element("xpath",f"//span[.='{self.region}']").click()
        time.sleep(0.5)
        self.browser.find_element("xpath","//*[@id='js-search-locations']/div/div[3]/div[2]/div[2]/div[5]").click()
        time.sleep(0.5)
        self.browser.find_element("xpath","//*[@id='js-search-filters']/div[2]/button").click()
        time.sleep(5)
        
    def get_id(self):
        ad_box = self.browser.find_element(By.CSS_SELECTOR, "#js-items-search > div.items_list")
        #print(allAdBox)
        all_ads = ad_box.find_elements(By.CLASS_NAME, "items-i")
        for ad in all_ads:
            ad_id = ad.get_attribute("data-item-id")
            self.id_set.add(ad_id)
            
    def showAddElements(self):
        print (self.id_set)
        
    def showElementCount(self):
        print(f"\rGötürülən elan id sayı: {len(self.id_set)}",end ="")
        return len(self.id_set)
        
        
    def navigate_page(self):
        while True:
            time.sleep(5)
            if self.showElementCount()>self.ad_count:
                break
            self.showElementCount()
            self.get_id()
            button_box = self.browser.find_element(By.CLASS_NAME, "pagination-inner").find_elements(By.TAG_NAME,"span")
            lastbuttonClass = button_box[len(button_box)-1].get_attribute("class")
            if(lastbuttonClass == "next"):
                button_box[len(button_box)-1].click()
            else:
                break
                
            
    def save_ad_information(self):
        
        for ad_id in self.id_set:
            checked = len(self.ad_info_list)
            print(f"\rGötürülən uyğun elan məlumatları sayı:{checked}",end ="")
            
            self.browser.get(f"https://bina.az/items/{ad_id}")
            time.sleep(1)
            current_url = self.browser.current_url
            if "yasayis-kompleksleri" in current_url:
                continue
            table_details = self.browser.find_elements(By.CSS_SELECTOR, "#js-item-show > div.item_show_content > div.side > div.parameters_section > div.param_info > table > tbody>tr")
            ad_info={}
            ad_info["ad_id"]=ad_id
            for row in table_details:
                    td_1 = row.find_elements(By.TAG_NAME,"td")[0].text
                    if td_1 =="Kateqoriya":
                        ad_cat = row.find_elements(By.TAG_NAME,"td")[1].text
                        ad_info["ad_cat"]=ad_cat
                    elif td_1 =="Mərtəbə":
                        ad_floor = row.find_elements(By.TAG_NAME,"td")[1].text
                        ad_info["add_floor"] =ad_floor
                    elif td_1 =="Sahə":
                        ad_square = row.find_elements(By.TAG_NAME,"td")[1].text
                        ad_info["ad_square"] =ad_square 
                    elif td_1 =="Otaq sayı":
                        ad_room = row.find_elements(By.TAG_NAME,"td")[1].text
                        ad_info["ad_room"] =ad_room
                    elif td_1 =="Çıxarış":
                        ad_doc = row.find_elements(By.TAG_NAME,"td")[1].text
                        ad_info["ad_doc"] =ad_doc
                    elif td_1 =="İpoteka":
                        ad_mortgage = row.find_elements(By.TAG_NAME,"td")[1].text
                        ad_info["ad_mortgage"] =ad_mortgage
                    elif td_1 =="Təmir":
                        ad_repair = row.find_elements(By.TAG_NAME,"td")[1].text
                        ad_info["ad_repair"] =ad_repair
                    else:
                        continue
                    
                    
                        
            ad_info["view_count"]=self.browser.find_element(By.CSS_SELECTOR,"#js-item-show > div.item_show_content > div.info > div.item_info > p:nth-child(2)").text
            ad_info["ad_time"]=self.browser.find_element(By.CSS_SELECTOR,"#js-item-show > div.item_show_content > div.info > div.item_info > p:nth-child(3)").text   
            ad_info["ad_address"]=self.browser.find_element(By.CSS_SELECTOR,"#js-item-show > div.item_show_content > div.side > div.parameters_section > div.map-link > section > div").text
            ad_info["ad_price"]=self.browser.find_element(By.CSS_SELECTOR,"#js-item-show > div.price_header > section > p > span.price-val").text
            ad_info["ad_price_currency"]=self.browser.find_element(By.CSS_SELECTOR,"#js-item-show > div.price_header > section > p > span.price-cur").text
            self.ad_info_list.append(ad_info)
            
                      
       

In [15]:
data =BinaAz(ads_type,estate_type,region,elan_sayi)
data.navigate_page()

Götürülən elan id sayı: 24

In [16]:
data.save_ad_information()

Götürülən uyğun elan məlumatları sayı:0

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#js-item-show > div.item_show_content > div.info > div.item_info > p:nth-child(2)"}
  (Session info: chrome=125.0.6422.113); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF776BB1F22+60322]
	(No symbol) [0x00007FF776B2CE99]
	(No symbol) [0x00007FF7769E7EBA]
	(No symbol) [0x00007FF776A37676]
	(No symbol) [0x00007FF776A3773C]
	(No symbol) [0x00007FF776A7E967]
	(No symbol) [0x00007FF776A5C25F]
	(No symbol) [0x00007FF776A7BC80]
	(No symbol) [0x00007FF776A5BFC3]
	(No symbol) [0x00007FF776A29617]
	(No symbol) [0x00007FF776A2A211]
	GetHandleVerifier [0x00007FF776EC946D+3301613]
	GetHandleVerifier [0x00007FF776F13693+3605267]
	GetHandleVerifier [0x00007FF776F09410+3563664]
	GetHandleVerifier [0x00007FF776C642F6+790390]
	(No symbol) [0x00007FF776B3

### Creating pandas dataframe from the scraped data

In [16]:
df = pd.DataFrame(data.ad_info_list,columns=['ad_id', 'ad_cat','add_floor','ad_square','ad_room','ad_price','ad_price_currency',
                                             'ad_doc','ad_mortgage','ad_repair','ad_address','view_count','ad_time'])

### Saving data to an excel file

In [18]:
#saving params
currentMinute = datetime.now().minute
currentHour = datetime.now().hour
currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year

In [19]:
# saving scraped data for analysis
df.to_excel(f"{currentYear}-{currentMonth}-{currentDay}-{currentHour}-{currentMinute}-{data.ads_type}_{data.estate_type}_{data.region}.xlsx")